# Helpful Functions

You will see these at the top of every module.  These are simply a set of reusable functions that we will make use of.  Each of them will be explained as the semester progresses.  They are explained in greater detail as the course progresses.  Class 4 contains a complete overview of these functions.

In [3]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low
        


# Chat Bots

Using the above code you can create your own primitive chat bots.  A some what famous video on Youtube from Cornell University shows what happens [when two chat bots converse](https://www.youtube.com/watch?v=WnzlbyTZsQY).  Other interesting chat bot type technology:

* [CleverBot](http://www.cleverbot.com/)
* [Computer Science Paper Generator](https://pdos.csail.mit.edu/archive/scigen/)

### Other Resources
* [Word Net](http://wordnet.princeton.edu/)
* [bAbI Datasets](https://research.fb.com/downloads/babi/)


# End-To-End Memory Networks

The origional source papers for End-to-End Memory Networks:

* Jason Weston, Antoine Bordes, Sumit Chopra, Tomas Mikolov, Alexander M. Rush, ["Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks"](http://arxiv.org/abs/1502.05698)
* Sainbayar Sukhbaatar, Arthur Szlam, Jason Weston, Rob Fergus, ["End-To-End Memory Networks"](http://arxiv.org/abs/1503.08895)

Other useful links for End-To-End Memory Networks

* [bAbI Datasets](https://research.fb.com/downloads/babi/)
* [Keras End-To-End Memory Networks](https://github.com/fchollet/keras/blob/master/examples/babi_memnn.py)
* [Online JavaScript Demo of End-to-End Memory Networks](http://yerevann.com/dmn-ui/#/)

#### Imports and Utility Functions

The following imports are needed to create the end-to-end memory network. Neither Keras nor TensorFlow directly support End-to-End Memory Networks (yet), so it is necessary to create them using existing tools.  Several functions are needed defined here to read the bAbI dataset that we are using to train.

In [4]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from functools import reduce
import pickle
import tarfile
import numpy as np
import re
import os
import time


def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)', sent) if x.strip()]


def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data):
    inputs, queries, answers = [], [], []
    for story, query, answer in data:
        inputs.append([word_idx[w] for w in story])
        queries.append([word_idx[w] for w in query])
        answers.append(word_idx[answer])
    return (pad_sequences(inputs, maxlen=story_maxlen),
            pad_sequences(queries, maxlen=query_maxlen),
            np.array(answers))


Using TensorFlow backend.


#### Getting the Data

The data is downloaded from the Internet, if needed.

As you can see (below), this dataset contains stories and questions about those stories.  The computer is not learning these specific stories below, but rather how to read a story and answer a question about that story.  Consider the first story "Mary moved to the bathroom. John went to the hallway." the computer is not learning that Mary is in the bathroom or John is in the hallway, this changes per story.  Rather, the computer is learning to parse the story and extract information about individual people and their locations.

The computer is learning to read, at least in a limited sense.

In [3]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))



11747328/11745123 [==============================] - 10s 1us/step
Extracting stories for the challenge: single_supporting_fact_10k


In [4]:
# See what the data looks like

for i in range(5):
    print("Story: {}".format(' '.join(train_stories[i][0])))
    print("Query: {}".format(' '.join(train_stories[i][1])))
    print("Answer: {}".format(train_stories[i][2]))
    print("---")

Story: Mary moved to the bathroom . John went to the hallway .
Query: Where is Mary ?
Answer: bathroom
---
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden .
Query: Where is Daniel ?
Answer: hallway
---
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom .
Query: Where is Daniel ?
Answer: hallway
---
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom . Mary moved to the hallway . Daniel travelled to the office .
Query: Where is Daniel ?
Answer: office
---
Story: Mary moved to the bathroom . John went to the hallway . Daniel went back to the hallway . Sandra moved to the garden . John moved to the office . Sandra journeyed to the bathroom . Mary mov

#### Building the Vocabulary

This type of neural network can only deal with a set vocabulary.  The words are indexed and each becomes a number.  Words not in the training vocabulary will not be recognized.

In [5]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')


for s in list(enumerate(vocab)):
    print(s)

-
Vocab size: 22 unique words
Story max length: 68 words
Query max length: 4 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom')
-
(0, '.')
(1, '?')
(2, 'Daniel')
(3, 'John')
(4, 'Mary')
(5, 'Sandra')
(6, 'Where')
(7, 'back')
(8, 'bathroom')
(9, 'bedroom')
(10, 'garden')
(11, 'hallway')
(12, 'is')
(13, 'journeyed')
(14, 'kitchen')
(15, 'moved')
(16, 'office')
(17, 'the')
(18, 'to')
(19, 'travelled')
(20, 'went')


#### Building the Training and Test Data

The training data that is actually sent to the neural network is the vectorized representation of the sentences.  Each word is replaced by its vocab number.  Additionally there are two parts to the input (x) data: story and query.  The answer (x) is a always a single vocab word number.  This is a classification network.  Any of the vocab words could potentially be the answer.  Stories can be at most 68 words and questions at most 4.  Both of these limits are automatically determined from the training data.

In [6]:
print('Vectorizing the word sequences...')
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')


Vectorizing the word sequences...
-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 4)
queries_test shape: (1000, 4)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000,)
answers_test shape: (1000,)
-


In [7]:
# See individual training element.

print("Story (x): {}".format(inputs_train[0]))
print("Question (x): {}".format(queries_train[0]))
print("Answer: {}".format(answers_train[0]))

Story (x): [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  5 16 19 18  9  1  4 21 19 18 12  1]
Question (x): [ 7 13  5  2]
Answer: 9


#### Compile the Neural Network

In [8]:
print('Compiling...')

# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print("Done.")


Compiling...
Done.


#### Train the Neural Network

It will take some time (probably up to 1/2 hour) to train this network on a CPU.  The network is saved.  If you've previously saved the neural network, you can skip this step and load the neural network in the next step.

In [9]:
start_time = time.time()
# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=120,
          validation_data=([inputs_test, queries_test], answers_test))

# save
save_path = "./dnn/"
# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(save_path,"chatbot.h5"))
# save the vocab too, indexes must be the same
pickle.dump( vocab, open( os.path.join(save_path,"vocab.pkl"), "wb" ) )

elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 6s 627us/step - loss: 1.9556 - acc: 0.1672 - val_loss: 1.7957 - val_acc: 0.1540
Epoch 2/120
10000/10000 [==============================] - 5s 490us/step - loss: 1.7296 - acc: 0.2432 - val_loss: 1.6453 - val_acc: 0.3320
Epoch 3/120
10000/10000 [==============================] - 5s 475us/step - loss: 1.6404 - acc: 0.3091 - val_loss: 1.5817 - val_acc: 0.3990
Epoch 4/120
10000/10000 [==============================] - 4s 443us/step - loss: 1.5495 - acc: 0.3894 - val_loss: 1.4681 - val_acc: 0.4560
Epoch 5/120
10000/10000 [==============================] - 5s 486us/step - loss: 1.4896 - acc: 0.4425 - val_loss: 1.4033 - val_acc: 0.5060
Epoch 6/120
10000/10000 [==============================] - 4s 438us/step - loss: 1.4388 - acc: 0.4691 - val_loss: 1.3535 - val_acc: 0.5010
Epoch 7/120
10000/10000 [==============================] - 5s 471us/step - loss: 1.4026 - acc: 0.4771 - val_loss: 1.3

10000/10000 [==============================] - 4s 407us/step - loss: 0.0718 - acc: 0.9743 - val_loss: 0.1329 - val_acc: 0.9640
Epoch 118/120
10000/10000 [==============================] - 4s 399us/step - loss: 0.0750 - acc: 0.9734 - val_loss: 0.1145 - val_acc: 0.9650
Epoch 119/120
10000/10000 [==============================] - 4s 397us/step - loss: 0.0833 - acc: 0.9716 - val_loss: 0.1170 - val_acc: 0.9600
Epoch 120/120
10000/10000 [==============================] - 4s 394us/step - loss: 0.0696 - acc: 0.9759 - val_loss: 0.1214 - val_acc: 0.9610


OSError: Unable to create file (unable to open file: name = './dnn/chatbot.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [10]:
# Load the model, if it exists, load vocab too
save_path = "./dnn/"
model = load_model(os.path.join(save_path,"chatbot.h5"))
vocab = pickle.load( open( os.path.join(save_path,"vocab.pkl"), "rb" ) )

#### Evaluate Accuracy

We evaluate the accuracy, using the same technique as previous classification networks.

In [11]:
pred = model.predict([inputs_test, queries_test])
# See what the predictions look like, they are just probabilities of each class.
print(pred)

[[  4.84025304e-19   4.20079538e-19   3.95324297e-19 ...,   4.96237592e-19
    4.13944029e-19   4.70353715e-19]
 [  4.13656544e-15   3.87317208e-15   3.58496912e-15 ...,   4.34152116e-15
    3.58241277e-15   4.05210271e-15]
 [  4.56332683e-21   4.07824639e-21   4.57121938e-21 ...,   5.32078113e-21
    4.45083794e-21   4.77842287e-21]
 ..., 
 [  2.28002502e-16   2.36599913e-16   2.75854623e-16 ...,   2.75745197e-16
    2.60833393e-16   2.55596268e-16]
 [  4.79898097e-18   4.69900296e-18   4.23174310e-18 ...,   5.21618440e-18
    4.59116590e-18   5.18658043e-18]
 [  3.16733639e-14   2.75788286e-14   2.90205753e-14 ...,   3.14569232e-14
    3.05475961e-14   3.46005742e-14]]


In [12]:
# Use argmax to turn those into actual predictions.  The class (word) with the highest
# probability is the answer.

pred = np.argmax(pred,axis=1)
print(pred)

[12  9 15 12 15 12 11 12 17 17 15 11 17 10 10 15 11 15 17 11 12 17 17  9 10
 10 10 15 15 15 12 17 15 15  9 17  9 11 15 10  9 11 12 12 11 15 12  9 17 10
 10 17 11 11 11 10 15 12 11 12 12 17  9 11 11 15 15  9 17 17 11 12 11 10  9
 15 15 15 12 17 11  9 10 15 17 11 11 12 15 15  9  9 12  9  9 15 15 15 10 10
 10  9 17 11 11 12 10  9 17 10  9  9 15 12 15 11 17 15  9 17 10  9  9 12 10
 17 11 11 11  9 11 11  9 11 10  9 17 15  9 15 17 12 12 17 10 15 15 15 15 15
 11 11  9 17 10  9 11 10 17 12 10 10 10 11  9 11 11 11 15 10 17 17 11  9  9
 17 15 10 17 17 12 17 12 12 10  9 17 11 15 17 12 15 11 12 11 11 17 15  9 10
 12 17 17 10 12  9 15 12  9  9  9 10 12 10 12 10  9 12 12  9  9 12 15 12 10
 17 12 11  9  9 17 11 12 17 11 10 10 10 10 10 12 15 11 11 15 15 11 11 12 11
 10 17  9  9 10 10 17 17 15 11 11 17 12 17 17  9 11 10  9 10 15 17 17 15  9
 17 15 12 17  9  9 11 17 17 11 17  9 17 11  9 15 15 10 15 15 11 11 12 12  9
  9  9 15 17 17 11  9 17 17 15 11 11 17 10 11 17 10 17 17 11 11 15 15 11 11
  9 10 11 11

In [13]:
score = metrics.accuracy_score(answers_test, pred)
print("Final accuracy: {}".format(score))

Final accuracy: 0.95


#### Adhoc Query

You might want to create your own stories and questions.  

In [14]:
print("Remember, I only know these words: {}".format(vocab))
print()
story = "Daniel went to the hallway. Mary went to the bathroom.  Daniel went to the bedroom."
query = "Where is Sandra?"

adhoc_stories = (tokenize(story), tokenize(query), '?')

adhoc_train, adhoc_query, adhoc_answer = vectorize_stories([adhoc_stories])

pred = model.predict([adhoc_train, adhoc_query])
print(pred[0])
pred = np.argmax(pred,axis=1)
print("Answer: {}({})".format(vocab[pred[0]-1],pred))


Remember, I only know these words: ['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'the', 'to', 'travelled', 'went']

[  2.87214818e-11   2.52614821e-11   2.65876106e-11   2.82483256e-11
   2.63113090e-11   2.84588603e-11   2.50336488e-11   2.90474068e-11
   2.89508313e-11   1.17750010e-02   3.78847151e-04   1.50052369e-01
   1.55859768e-01   2.86468557e-11   3.12608307e-11   6.80781960e-01
   2.86887406e-11   1.15210935e-03   2.71236419e-11   2.95860975e-11
   2.71023360e-11   3.10942071e-11]
Answer: kitchen([15])


# Word2Vec

Word2vec is a group of related models that are used to produce word embeddings. These models are shallow, two-layer neural networks that are trained to reconstruct linguistic contexts of words. Word2vec takes as its input a large corpus of text and produces a vector space, typically of several hundred dimensions, with each unique word in the corpus being assigned a corresponding vector in the space. Word vectors are positioned in the vector space such that words that share common contexts in the corpus are located in close proximity to one another in the space.

Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). [Efficient estimation of word representations in vector space](https://arxiv.org/abs/1301.3781). arXiv preprint arXiv:1301.3781.

![Word2Vec](https://pbs.twimg.com/media/C7jJxIjWkAA8E_s.jpg)
[Trust Word2Vec](https://twitter.com/DanilBaibak/status/844647217885581312)

### Suggested Software for Word2Vec

* [GoogleNews Vectors](https://code.google.com/archive/p/word2vec/), [GitHub Mirror](https://github.com/mmihaltz/word2vec-GoogleNews-vectors)
* [Python Gensim](https://radimrehurek.com/gensim/)


In [5]:
try:
    path = get_file('GoogleNews-vectors-negative300.bin.gz', origin='https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz')
except:
    print('Error downloading')
    raise
    
print(path)    


C:\Users\sureshkm\.keras\datasets\GoogleNews-vectors-negative300.bin.gz


In [6]:
import gensim

# Not that the path below refers to a location on my hard drive.
# You should download GoogleNews Vectors (see suggested software above)
model = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

C:\Users\sureshkm\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Word2vec makes each word a vector.  We are using the 300-number vector, which can be seen for the word "hello".

In [7]:
w = model['hello']

In [8]:
print(len(w))

300


In [9]:
print(w)

[-0.05419922  0.01708984 -0.00527954  0.33203125 -0.25       -0.01397705
 -0.15039062 -0.265625    0.01647949  0.3828125  -0.03295898 -0.09716797
 -0.16308594 -0.04443359  0.00946045  0.18457031  0.03637695  0.16601562
  0.36328125 -0.25585938  0.375       0.171875    0.21386719 -0.19921875
  0.13085938 -0.07275391 -0.02819824  0.11621094  0.15332031  0.09082031
  0.06787109 -0.0300293  -0.16894531 -0.20800781 -0.03710938 -0.22753906
  0.26367188  0.012146    0.18359375  0.31054688 -0.10791016 -0.19140625
  0.21582031  0.13183594 -0.03515625  0.18554688 -0.30859375  0.04785156
 -0.10986328  0.14355469 -0.43554688 -0.0378418   0.10839844  0.140625
 -0.10595703  0.26171875 -0.17089844  0.39453125  0.12597656 -0.27734375
 -0.28125     0.14746094 -0.20996094  0.02355957  0.18457031  0.00445557
 -0.27929688 -0.03637695 -0.29296875  0.19628906  0.20703125  0.2890625
 -0.20507812  0.06787109 -0.43164062 -0.10986328 -0.2578125  -0.02331543
  0.11328125  0.23144531 -0.04418945  0.10839844 -0.28

The code below shows the distance between two words.

In [10]:
import numpy as np

w1 = model['vapor']
w2 = model['water']

dist = np.linalg.norm(w1-w2)

print(dist)

3.57513


This shows the classic word2vec equation of **queen = (king - man) + female**

In [11]:
model.most_similar(positive=['lion','cat'], negative=['dogs'])


[('gorilla', 0.4916578531265259),
 ('tiger', 0.47669246792793274),
 ('elephant', 0.47190138697624207),
 ('monkey', 0.45950955152511597),
 ('squirrel', 0.4441283941268921),
 ('frog', 0.4412733316421509),
 ('lions', 0.43587300181388855),
 ('Nile_crocodile', 0.42671990394592285),
 ('rabbit', 0.42445677518844604),
 ('giraffe', 0.42444396018981934)]

The following code shows which item does not belong with the others.

In [12]:
model.doesnt_match("house garage store dog".split())


'dog'

The following code shows the similarity between two words.

In [13]:
model.similarity('lambo', 'ferrari')

0.62627882321344674

The following code shows which words are most similar to the given one.

In [24]:
# %%timeit
model.most_similar("newt")

[('newts', 0.5993279218673706),
 ('frog', 0.5425999164581299),
 ('salamander', 0.5349865555763245),
 ('frogs', 0.5226622223854065),
 ('lizard', 0.5167595148086548),
 ('salamanders', 0.5082288980484009),
 ('squirrel', 0.4951688051223755),
 ('otter', 0.49366065859794617),
 ('toad', 0.4875979423522949),
 ('snails', 0.48703545331954956)]

# More on LSTM

* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* [LSTM Music](https://www.youtube.com/watch?v=0VTI1BBLydE)
* [Natural Language Processing from Scratch](https://arxiv.org/abs/1103.0398)